In [1]:
import sys
sys.path.append('C:\\Users\\DavidB\\PycharmProjects\\My_Football_Analytics\\Basics')
sys.path.append('C://Users//DavidB//PycharmProjects//My_Football_Analytics//Position_data')

import pandas as pd
import numpy as np
import torch
from torch.nn.functional import softplus
import Data.Metrica_IO as mio
from Tracking_Data import tracking_data

## set GPU or CPU (uncomment second line to use CPU)
# device = torch.device('cuda:0'); dtype = torch.float16
device = 'cpu'; dtype = torch.float32




In [2]:
data_dir = 'sample-data-master/data/'
game_id = 1
away = mio.tracking_data(DATADIR=data_dir, game_id=game_id, teamname='Away')
home = mio.tracking_data(DATADIR=data_dir, game_id=game_id, teamname='Home')
td = mio.merge_tracking_data(home, away)
td_object = tracking_data(td, data_source='metrica', scale_to_pitch='myPitch')
td_object.get_velocities()

td_object.data

Reading team: away
Reading team: home


,Period,Time [s],Home_11_x,Home_11_y,Home_1_x,Home_1_y,Home_2_x,Home_2_y,Home_3_x,Home_3_y,...,Home_6_speed,Home_7_vx,Home_7_vy,Home_7_speed,Home_8_vx,Home_8_vy,Home_8_speed,Home_9_vx,Home_9_vy,Home_9_speed
Frame,,,,,,,,,,,,,,,,,,,,,
1,1,0.04,0.08610,33.64530,34.28040,22.54070,35.38605,33.23905,32.47335,41.90615,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,1,0.08,0.10080,33.64530,34.28040,22.54070,35.38605,33.23905,32.47335,41.90615,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,1,0.12,0.11970,33.64530,34.28040,22.54070,35.38605,33.23905,32.47335,41.90615,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,1,0.16,0.12705,33.64530,34.25310,22.54395,35.37135,33.15780,32.49120,41.88990,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,1,0.20,0.13545,33.64530,34.22685,22.57515,35.34720,33.13830,32.49540,41.90680,...,0.77967,0.585,1.309286,1.434034,-0.04875,1.727143,1.727831,0.206250,1.239643,1.256684
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
145002,2,5800.08,9.85530,35.51925,NaN,NaN,69.13515,23.01130,72.21060,21.11655,...,NaN,0.000,0.000000,0.000000,0.37125,-0.162500,0.405256,-0.277500,0.526964,0.595565
145003,2,5800.12,9.92250,35.51925,NaN,NaN,69.16665,23.02105,72.20850,21.10615,...,NaN,0.000,0.000000,0.000000,0.48375,-0.218214,0.530690,0.037500,0.698750,0.699756
145004,2,5800.16,9.98340,35.51925,NaN,NaN,69.20655,23.03015,72.22530,21.12435,...,NaN,0.000,0.000000,0.000000,0.64875,-0.349955,0.737120,0.391875,0.777679,0.870833


In [3]:
# ball, and both team's position data in seperate data frames
Home = td_object.get_team('Home', selection='position', T_P=False)
Away = td_object.get_team('Away', selection='position', T_P=False)
Ball = td_object.get_ball(pos_only=True)

# velcoties as well

vel_Home = td_object.get_team('Home', selection='velocity', T_P=False)
vel_Away = td_object.get_team('Away', selection='velocity', T_P=False)


In [8]:
# both team's position data, velocities and ball to array

jitter = 1e-12 ## prevents division by zero when player is stationary



def pos_to_array(pos_data, nan_to=-1000, ball=False):
    if 'Period' in pos_data.columns or 'Time [s]' in pos_data.columns:
        raise ValueError('Data should include positon data only. Not any other columns!')
    n_players = int(len(pos_data.columns)/2)
    if ball==True:
        array = np.asarray(pos_data.iloc[:,range(0, 2)])[:,None,None,:]
    else:
        array = np.array([np.asarray(pos_data.iloc[:,range(j*2, j*2 +2)]) for j in range(n_players)])
    
    np.nan_to_num(array,copy=False,nan=nan_to)
    return array

A_Ball = pos_to_array(Ball, nan_to=np.nan, ball=True)
A_Home = pos_to_array(Home)
A_Away = pos_to_array(Away)
A_vel_Home = pos_to_array(vel_Home) + jitter
A_vel_Away = pos_to_array(vel_Away) + jitter

# severe changes in the shape of the arrays
# I am not 100% how and why this is necessary
# This is probably necessary for later 
A_Home = A_Home[:,:,None,None,:]
A_Away = A_Away[:,:,None,None,:]
A_vel_Home = A_vel_Home[:,:,None,None,:]
A_vel_Away = A_vel_Away[:,:,None,None,:]
A_Ball = A_Ball[None,:]

print(A_Home.shape, A_Away.shape, A_Ball.shape, A_vel_Away.shape, A_vel_Home.shape)




(14, 145006, 1, 1, 2) (14, 145006, 1, 1, 2) (1, 145006, 1, 1, 2) (14, 145006, 1, 1, 2) (14, 145006, 1, 1, 2)


In [12]:
## set up evaluation grid and set some pitch control parameters (these are taken from the FoT code)
reaction_time = 0.7
max_player_speed = 5.
average_ball_speed = 15.
sigma = np.pi / np.sqrt(3.) / 0.45
lamb = 4.3
n_grid_points_x = 50
n_grid_points_y = 30
# create grid based on tensors
XX,YY = torch.meshgrid(torch.linspace(0,105,n_grid_points_x, device = device, dtype=dtype),
                       torch.linspace(0,68,n_grid_points_y,device=device,dtype=dtype))
target_position = torch.stack([XX,YY],2)[None,None,:,:,:] # all possible positions

print(XX.shape, YY.shape, target_position.shape)

torch.Size([50, 30]) torch.Size([50, 30]) torch.Size([1, 1, 50, 30, 2])


In [14]:
# the weights and the x-points of the Gauss–Legendre quadrature are set up and stored as torch tensors
# I assume that, as the Gauss–Legendre quadrature spans the range of [-1, 1] (length of 2), the 50 points selected
# represent the number of frames in a 2 second time period based on a framerate of 25Hz (i.e. 25 frames / second)
ti,wi = np.polynomial.legendre.leggauss(50) ## used for numerical integration later on
ti = torch.tensor(ti,device = device,dtype=dtype)
wi = torch.tensor(wi,device=device,dtype=dtype)


In [35]:
n_frames = A_Home.shape[1]
first_frame = 5
batch_size = 250

In [36]:
# time to intercept empty torch
tti = torch.empty([28,batch_size,n_grid_points_x,n_grid_points_y],device = device,dtype=dtype)
# 28 players*500 batches*grid
tmp2 = torch.empty([28,batch_size,n_grid_points_x,n_grid_points_y,1],device = device,dtype=dtype)
# 28 players*500 batches*grid * 1
pc = torch.empty([n_frames,n_grid_points_x,n_grid_points_y],device = device,dtype=dtype)
# frames * grid


In [37]:
n_frames = 250 # for speed just the first batch

for f in range(int(n_frames/batch_size)):
    bp = torch.tensor(A_Ball[:,(first_frame + f*batch_size):(np.minimum(first_frame + (f+1)*batch_size,
                                                                          int(first_frame + n_frames)))],
                      device = device,dtype=dtype)
    hp = torch.tensor(A_Home[:,(first_frame + f*batch_size):(np.minimum(first_frame + (f+1)*batch_size,
                                                                          int(first_frame + n_frames)))],
                      device = device,dtype=dtype)
    hv = torch.tensor(A_vel_Home[:,(first_frame + f*batch_size):(np.minimum(first_frame + (f+1)*batch_size,
                                                                        int(first_frame + n_frames)))],
                      device = device,dtype=dtype)
    ap = torch.tensor(A_Away[:,(first_frame + f*batch_size):(np.minimum(first_frame + (f+1)*batch_size,
                                                                          int(first_frame + n_frames)))],
                      device = device,dtype=dtype)
    av = torch.tensor(A_vel_Away[:,(first_frame + f*batch_size):(np.minimum(first_frame + (f+1)*batch_size,
                                                                        int(first_frame + n_frames)))],
                      device = device,dtype=dtype)
    
    print(bp.shape, hp.shape, hv.shape, ap.shape, av.shape)
    
    ball_travel_time = torch.norm(target_position - bp, dim=4).div_(average_ball_speed) 
    # ball travel time to each location in each frame in the batch
    
    r_reaction_home = hp + hv.mul_(reaction_time) # position after reaction time (vector)
    r_reaction_away = ap + av.mul_(reaction_time) # = position + velocity multiplied by reaction time
    r_reaction_home = r_reaction_home - target_position # distance to target position (vector)
    r_reaction_away = r_reaction_away - target_position # after reaction time
    
     # time to intercept for home and away filled 
    # torch.norm --> distance of position in grid to target position from vector to value
    # add reaction time to time of intercept
    # divide distance by speed --> time of intercept
    # i think it should be the other way around! Divide first and then add reaction time
    tti[:14,:ball_travel_time.shape[1]] = torch.norm(r_reaction_home,dim=4).add_(reaction_time).div_(max_player_speed)
    tti[14:,:ball_travel_time.shape[1]] = torch.norm(r_reaction_away,dim=4).add_(reaction_time).div_(max_player_speed)
    # tti[:14,:ball_travel_time.shape[1]] = torch.norm(r_reaction_home,dim=4).div_(max_player_speed).add_(reaction_time)
    # tti[14:,:ball_travel_time.shape[1]] = torch.norm(r_reaction_away,dim=4).div_(max_player_speed).add_(reaction_time)

    tmp2[...,0] = sigma * (ball_travel_time - tti)
    
    tmp1 = sigma * 0.5 * (ti + 1) * 10 + tmp2
    
    # every players (scaled) time to control fed into a sigmoid function
    # --> probabilites [0,1], multiplied by lambda (default 4.3) --> range of [0, 4.3]
    hh = torch.sigmoid(tmp1[:14]).mul_(4.3)
    # teams intercept probability as the sum of players' control probabilities
    h = hh.sum(0)
    
    S = torch.exp(-lamb*torch.sum(softplus(tmp1) - softplus(tmp2),dim=0).div_(sigma))

    # numerische Integralbildung durch Gewichte (wi) durch Matrixvektorprukt aus Sh und den Gewichten 
    # i.e. Gewicht wird mit dem Entsprechenden der 50 Werte aus der Verteilung multipliziert
    pc[(first_frame + f*batch_size):(np.minimum(first_frame + (f+1)*batch_size,int(first_frame + n_frames)))] = torch.matmul(S*h,wi).mul_(5.)


torch.Size([1, 250, 1, 1, 2]) torch.Size([14, 250, 1, 1, 2]) torch.Size([14, 250, 1, 1, 2]) torch.Size([14, 250, 1, 1, 2]) torch.Size([14, 250, 1, 1, 2])


In [38]:
# not working yet!


pc

tensor([[[0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         ...,
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.]],

        [[0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         ...,
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.]],

        [[0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         ...,
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.]],

        ...,

        [[0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         ...,
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0., 

In [31]:
print(f'Minimum of tmp2: {tmp2.min()}, Maximum of tmp2: {tmp2.max()}, shape: {tmp2.shape}')
# --> + [0.0228, 40.02838] = time to intercept (scaled by sigma)
print(f'Minimum of tmp1: {tmp1.min()}, Maximum of tmp1: {tmp1.max()}, shape: {tmp1.shape}')
# tmp1 =  distribution of possible times to intercept available (scaled by sigma)
# tmp2 =  single estimate time to intercept (scaled by sigma) = exponent of intercept probability equation (3)

Minimum of tmp2: -2024.591552734375, Maximum of tmp2: 16.065284729003906, shape: torch.Size([28, 250, 50, 30, 1])
Minimum of tmp1: -2024.5687255859375, Maximum of tmp1: 56.34909439086914, shape: torch.Size([28, 250, 50, 30, 50])


In [32]:
S = torch.exp(-lamb*torch.sum(softplus(tmp1) - softplus(tmp2),dim=0).div_(sigma))
print(f'Minimum of S: {S.min()}, Maximum of S: {S.max()}, shape: {S.shape}')
# transforms the whole thing to a range of [0, 1]
# = control probability -1 based on all possible times to control?! 

Minimum of S: 0.0, Maximum of S: 1.0, shape: torch.Size([250, 50, 30, 50])


In [40]:
hh = torch.sigmoid(tmp1[:14]).mul_(4.3)
print(f'Minimum of hh: {hh.min()}, Maximum of hh: {hh.max()}, shape: {hh.shape}')
# converts time available to intercept the ball into a probability [0,1] --> hints at intercept probability
# sigmoid with tmp1 as the exponent is exactly the intercept probability function (3)
# why multiply by lambda?! Makes no sense to me
# only for the home team!

# teams intercept probability as the sum of players' intercept probabilities
h = hh.sum(0)
print(f'Minimum of h: {h.min()}, Maximum of h: {h.max()}, shape: {h.shape}')


Minimum of hh: 0.0, Maximum of hh: 4.300000190734863, shape: torch.Size([14, 250, 50, 30, 50])
Minimum of h: 3.443734737444118e-16, Maximum of h: 34.39999771118164, shape: torch.Size([250, 50, 30, 50])


In [39]:
hh

tensor([[[[[3.6052e-04, 3.9741e-04, 4.7335e-04,  ..., 4.3000e+00,
            4.3000e+00, 4.3000e+00],
           [1.6870e-03, 1.8596e-03, 2.2148e-03,  ..., 4.3000e+00,
            4.3000e+00, 4.3000e+00],
           [8.0054e-03, 8.8229e-03, 1.0505e-02,  ..., 4.3000e+00,
            4.3000e+00, 4.3000e+00],
           ...,
           [4.7744e-03, 5.2623e-03, 6.2666e-03,  ..., 4.3000e+00,
            4.3000e+00, 4.3000e+00],
           [9.2287e-04, 1.0173e-03, 1.2116e-03,  ..., 4.3000e+00,
            4.3000e+00, 4.3000e+00],
           [1.8173e-04, 2.0033e-04, 2.3861e-04,  ..., 4.3000e+00,
            4.3000e+00, 4.3000e+00]],

          [[2.1187e-04, 2.3355e-04, 2.7818e-04,  ..., 4.3000e+00,
            4.3000e+00, 4.3000e+00],
           [9.7848e-04, 1.0786e-03, 1.2847e-03,  ..., 4.3000e+00,
            4.3000e+00, 4.3000e+00],
           [4.5821e-03, 5.0504e-03, 6.0143e-03,  ..., 4.3000e+00,
            4.3000e+00, 4.3000e+00],
           ...,
           [2.6157e-03, 2.8832e-03, 3.4

In [41]:
S

tensor([[[[1.0000e+00, 9.9999e-01, 9.9997e-01,  ..., 0.0000e+00,
           0.0000e+00, 0.0000e+00],
          [9.9999e-01, 9.9995e-01, 9.9986e-01,  ..., 0.0000e+00,
           0.0000e+00, 0.0000e+00],
          [9.9996e-01, 9.9975e-01, 9.9933e-01,  ..., 0.0000e+00,
           0.0000e+00, 0.0000e+00],
          ...,
          [9.9997e-01, 9.9985e-01, 9.9960e-01,  ..., 0.0000e+00,
           0.0000e+00, 0.0000e+00],
          [9.9999e-01, 9.9997e-01, 9.9992e-01,  ..., 0.0000e+00,
           0.0000e+00, 0.0000e+00],
          [1.0000e+00, 9.9999e-01, 9.9998e-01,  ..., 0.0000e+00,
           0.0000e+00, 0.0000e+00]],

         [[1.0000e+00, 9.9999e-01, 9.9998e-01,  ..., 0.0000e+00,
           0.0000e+00, 0.0000e+00],
          [9.9999e-01, 9.9997e-01, 9.9992e-01,  ..., 0.0000e+00,
           0.0000e+00, 0.0000e+00],
          [9.9997e-01, 9.9986e-01, 9.9962e-01,  ..., 0.0000e+00,
           0.0000e+00, 0.0000e+00],
          ...,
          [9.9999e-01, 9.9992e-01, 9.9978e-01,  ..., 0.0000